In [ ]:
#| hide
from trouver.helper import *

# trouver

> Create and maintain mathematical Obsidian.md notes, and gather data from them to train ML models

This file will become your README and also the index of your documentation.

In [ ]:
# TODO Write front description

## Install

In [ ]:
# TODO Write installation instructions

```sh
pip install trouver
```

## How to use

In [ ]:
# TODO Write basic usage

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2

## How the examples are structured

TODO: add how the `nbs` folder has a `_tests` folder.